<a href="https://colab.research.google.com/github/hansolko/newsEmail/blob/master/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 52.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=0fa75eae7ec7002f342586c1fa2a263ca7f6bae75857677e37e3984946be1ea4
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from datetime import datetime, date
from pyspark.sql.types import DateType
from pyspark.sql.functions import col, udf, when, count, countDistinct

# Day 1.

## Q1. 데이터 생성

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples').getOrCreate()

In [ ]:
data = [("James", "M", 60000), ("Michale", "M", 70000),
        ("Robert", None, 400000), ("Maria", "F", 500000),
        ("Jen", "", None)]
columns = ["name", "gender", 'salary']
df = spark.createDataFrame(data = data, schema = columns)
df.show()

+-------+------+------+
|   name|gender|salary|
+-------+------+------+
|  James|     M| 60000|
|Michale|     M| 70000|
| Robert|  null|400000|
|  Maria|     F|500000|
|    Jen|      |  null|
+-------+------+------+



In [ ]:
data2 = [("MEM001", "M", "2021-03-17"),
        ("MEM003", "F", "2021-11-01"),
        ("MEM002", "M", "2021-05-20")]
columns2 = ["name", "gender", 'date']
df2 = spark.createDataFrame(data = data2, schema = columns2)
df2.show()

+------+------+----------+
|  name|gender|      date|
+------+------+----------+
|MEM001|     M|2021-03-17|
|MEM003|     F|2021-11-01|
|MEM002|     M|2021-05-20|
+------+------+----------+



## Q2. 데이터 타입 변환 (datetime type으로)

In [ ]:
# This function converts the string cell into a date:
func =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())

df3 = df2.withColumn('datetime', func(col('date')))
df3.show()
df3.printSchema()

+------+------+----------+----------+
|  name|gender|      date|  datetime|
+------+------+----------+----------+
|MEM001|     M|2021-03-17|2021-03-17|
|MEM003|     F|2021-11-01|2021-11-01|
|MEM002|     M|2021-05-20|2021-05-20|
+------+------+----------+----------+

root
 |-- name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- date: string (nullable = true)
 |-- datetime: date (nullable = true)



In [ ]:
# This function makes month column by using datetime column
func2 = udf(lambda x: x.month)

df4 = df3.withColumn('month', func2(col('datetime')))
df5 = df4.drop(df4.datetime)   # 사용한 열 삭제
df5.show()

+------+------+----------+-----+
|  name|gender|      date|month|
+------+------+----------+-----+
|MEM001|     M|2021-03-17|    3|
|MEM003|     F|2021-11-01|   11|
|MEM002|     M|2021-05-20|    5|
+------+------+----------+-----+



# Day 2.

## Q1. 데이터 생성

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples').getOrCreate()

In [ ]:
day2_data = [("MEM001", 10, "2021-03-17", 12), 
             ("MEM003", 20, "2021-11-01", 1),
             ("MEM002", 10, "2021-05-20", 45), 
             ("MEM00", 10, "2021-05-20", 32),
             ("MEM00", 10, "2021-05-20", 55),
             ("MEM00", 10, "2021-05-20", 12), 
             ("MEM0", 20, "2021-05-20", 1),
             ("MEM0", 10, "2021-05-20", 45), 
             ("MEM00", 30, "2021-05-20", 32),
             ("MEM00", None, "2021-05-20", 23),
             ("MEM001", 10, "2021-03-17", 23), 
             ("MEM003", 20, "2021-11-01", 12),
             ("MEM002", 10, "2021-05-20", 32), 
             ("MEM00", 30, "2021-05-20", 32),
             ("MEM00", 10, "2021-05-20", 55),
             ("MEM00", 20, "2021-05-20", None), 
             ("MEM0", 20, "2021-05-20", 1),
             ("MEM0", 10, "2021-05-20", 54), 
             ("MEM00", 10, "2021-05-20", 3),
             ("MEM00", 20, "2021-05-20", 53)]
day2_columns = ["회원ID", "gender", 'date', 'age']
day2_df = spark.createDataFrame(data = day2_data, schema = day2_columns)
day2_df.show()

+------+------+----------+----+
|회원ID|gender|      date| age|
+------+------+----------+----+
|MEM001|    10|2021-03-17|  12|
|MEM003|    20|2021-11-01|   1|
|MEM002|    10|2021-05-20|  45|
| MEM00|    10|2021-05-20|  32|
| MEM00|    10|2021-05-20|  55|
| MEM00|    10|2021-05-20|  12|
|  MEM0|    20|2021-05-20|   1|
|  MEM0|    10|2021-05-20|  45|
| MEM00|    30|2021-05-20|  32|
| MEM00|  null|2021-05-20|  23|
|MEM001|    10|2021-03-17|  23|
|MEM003|    20|2021-11-01|  12|
|MEM002|    10|2021-05-20|  32|
| MEM00|    30|2021-05-20|  32|
| MEM00|    10|2021-05-20|  55|
| MEM00|    20|2021-05-20|null|
|  MEM0|    20|2021-05-20|   1|
|  MEM0|    10|2021-05-20|  54|
| MEM00|    10|2021-05-20|   3|
| MEM00|    20|2021-05-20|  53|
+------+------+----------+----+



## Q2. 사용자 함수 생성 후, 컬럼에 적용 (lambda)

In [ ]:
# Q2
def new_gender_generator(x):
    if x == 10:
        return "여"
    elif x == 20:
        return "남"
    else:
        return "정보없음"

def new_age_generator(x):
    if x == None:
        return "정보없음"
    elif x < 10:
        return "정보없음"
    elif x < 20:
        return "10대"
    elif x < 30:
        return "20대"
    elif x < 40:
        return "30대"
    else:
        return "40대 이상"


gender_udf =  udf(lambda x: new_gender_generator(x))
age_udf =  udf(lambda x: new_age_generator(x))

# 적용
day2_df2 = day2_df.withColumn('new_gender', gender_udf(col('gender')))
day2_df3 = day2_df2.withColumn('new_age', age_udf(col('age')))

day2_df3.show()
day2_df3.printSchema()

+------+------+----------+----+----------+---------+
|회원ID|gender|      date| age|new_gender|  new_age|
+------+------+----------+----+----------+---------+
|MEM001|    10|2021-03-17|  12|        여|     10대|
|MEM003|    20|2021-11-01|   1|        남| 정보없음|
|MEM002|    10|2021-05-20|  45|        여|40대 이상|
| MEM00|    10|2021-05-20|  32|        여|     30대|
| MEM00|    10|2021-05-20|  55|        여|40대 이상|
| MEM00|    10|2021-05-20|  12|        여|     10대|
|  MEM0|    20|2021-05-20|   1|        남| 정보없음|
|  MEM0|    10|2021-05-20|  45|        여|40대 이상|
| MEM00|    30|2021-05-20|  32|  정보없음|     30대|
| MEM00|  null|2021-05-20|  23|  정보없음|     20대|
|MEM001|    10|2021-03-17|  23|        여|     20대|
|MEM003|    20|2021-11-01|  12|        남|     10대|
|MEM002|    10|2021-05-20|  32|        여|     30대|
| MEM00|    30|2021-05-20|  32|  정보없음|     30대|
| MEM00|    10|2021-05-20|  55|        여|40대 이상|
| MEM00|    20|2021-05-20|null|        남| 정보없음|
|  MEM0|    20|2021-05-20|   1|        남| 정보없음|
|  ME

# Day3.

## Q1. 데이터 생성

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples').getOrCreate()

In [ ]:
day3_data = [("MEM001","2021-03-17"), 
             ("MEM003","2021-11-01"),
             ("MEM002","2021-05-20"), 
             ("MEM00","2021-05-20")]
day3_columns = ["회원ID", 'date']
day3_df = spark.createDataFrame(data = day3_data, schema = day3_columns)
day3_df.show()

+------+----------+
|회원ID|      date|
+------+----------+
|MEM001|2021-03-17|
|MEM003|2021-11-01|
|MEM002|2021-05-20|
| MEM00|2021-05-20|
+------+----------+



## Q2. INNER JOIN TEST

In [ ]:
day2_df3.show()
day3_df.show()

+------+------+----------+----+----------+---------+
|회원ID|gender|      date| age|new_gender|  new_age|
+------+------+----------+----+----------+---------+
|MEM001|    10|2021-03-17|  12|        여|     10대|
|MEM003|    20|2021-11-01|   1|        남| 정보없음|
|MEM002|    10|2021-05-20|  45|        여|40대 이상|
| MEM00|    10|2021-05-20|  32|        여|     30대|
| MEM00|    10|2021-05-20|  55|        여|40대 이상|
| MEM00|    10|2021-05-20|  12|        여|     10대|
|  MEM0|    20|2021-05-20|   1|        남| 정보없음|
|  MEM0|    10|2021-05-20|  45|        여|40대 이상|
| MEM00|    30|2021-05-20|  32|  정보없음|     30대|
| MEM00|  null|2021-05-20|  23|  정보없음|     20대|
|MEM001|    10|2021-03-17|  23|        여|     20대|
|MEM003|    20|2021-11-01|  12|        남|     10대|
|MEM002|    10|2021-05-20|  32|        여|     30대|
| MEM00|    30|2021-05-20|  32|  정보없음|     30대|
| MEM00|    10|2021-05-20|  55|        여|40대 이상|
| MEM00|    20|2021-05-20|null|        남| 정보없음|
|  MEM0|    20|2021-05-20|   1|        남| 정보없음|
|  ME

In [ ]:
# join test (사실 할 필요 없음)
inner_join = day2_df3.join(day3_df, day2_df3.date == day3_df.date, how = 'inner')
inner_join.show()

+------+------+----------+---+----------+---------+------+----------+
|회원ID|gender|      date|age|new_gender|  new_age|회원ID|      date|
+------+------+----------+---+----------+---------+------+----------+
|MEM001|    10|2021-03-17| 12|        여|     10대|MEM001|2021-03-17|
|MEM001|    10|2021-03-17| 23|        여|     20대|MEM001|2021-03-17|
|MEM002|    10|2021-05-20| 45|        여|40대 이상|MEM002|2021-05-20|
|MEM002|    10|2021-05-20| 45|        여|40대 이상| MEM00|2021-05-20|
| MEM00|    10|2021-05-20| 32|        여|     30대|MEM002|2021-05-20|
| MEM00|    10|2021-05-20| 32|        여|     30대| MEM00|2021-05-20|
| MEM00|    10|2021-05-20| 55|        여|40대 이상|MEM002|2021-05-20|
| MEM00|    10|2021-05-20| 55|        여|40대 이상| MEM00|2021-05-20|
| MEM00|    10|2021-05-20| 12|        여|     10대|MEM002|2021-05-20|
| MEM00|    10|2021-05-20| 12|        여|     10대| MEM00|2021-05-20|
|  MEM0|    20|2021-05-20|  1|        남| 정보없음|MEM002|2021-05-20|
|  MEM0|    20|2021-05-20|  1|        남| 정보없음| MEM00|2021

In [ ]:
# groupby
inner_join_by_gender_age = inner_join.groupby(["new_gender", "new_age"]).count()
inner_join_by_gender_age.show()

+----------+---------+-----+
|new_gender|  new_age|count|
+----------+---------+-----+
|        남|     10대|    1|
|        여| 정보없음|    2|
|        여|     20대|    1|
|  정보없음|     20대|    2|
|        남| 정보없음|    7|
|        여|     30대|    4|
|        남|40대 이상|    2|
|        여|40대 이상|   10|
|  정보없음|     30대|    4|
|        여|     10대|    3|
+----------+---------+-----+



## Q3. 열 이름 수정

In [ ]:
# 열 이름 '활동고객수'로 변경
df_spark = inner_join_by_gender_age.select(
    col('new_gender'),
    col('new_age'),
    col('count').alias('활동고객수') 
)
df_spark.show()

+----------+---------+----------+
|new_gender|  new_age|활동고객수|
+----------+---------+----------+
|        남|     10대|         1|
|        여| 정보없음|         2|
|        여|     20대|         1|
|  정보없음|     20대|         2|
|        남| 정보없음|         7|
|        여|     30대|         4|
|        남|40대 이상|         2|
|        여|40대 이상|        10|
|  정보없음|     30대|         4|
|        여|     10대|         3|
+----------+---------+----------+



## Q4. Sorting

In [ ]:
# sorting
sort_by_gender = df_spark.sort(df_spark.new_gender.asc())
sort_by_gender_age = sort_by_gender.sort(sort_by_gender.new_age.asc())
sort_by_gender_age.show()

+----------+---------+----------+
|new_gender|  new_age|활동고객수|
+----------+---------+----------+
|        남|     10대|         1|
|        여|     10대|         3|
|        여|     20대|         1|
|  정보없음|     20대|         2|
|        여|     30대|         4|
|  정보없음|     30대|         4|
|        여|40대 이상|        10|
|        남|40대 이상|         2|
|        여| 정보없음|         2|
|        남| 정보없음|         7|
+----------+---------+----------+



# Day4.

## Q1. csv 파일 불러오기 & 월별로 회원수 Count하기

In [3]:
import os
os.getcwd()

'/content'

In [42]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

# raw data
df = spark.read.csv("OnlineRetailClean_spark.csv",header=True)
df.show()

+---+------+-------+--------------------+--------+--------------+---------+----------+--------------+-------------+------------+
|_c0|MBR_ID|MPIC_ID|         Description|Quantity|  ADMS_DTIME_M|UnitPrice|CustomerID|       Country|CheckoutPrice|CRET_DTIME_M|
+---+------+-------+--------------------+--------+--------------+---------+----------+--------------+-------------+------------+
|  0|536365| 85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|         15.3|  2011-02-15|
|  1|536365|  71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|        20.34|  2010-12-22|
|  2|536365| 84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|           22|  2011-03-19|
|  3|536365| 84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|        20.34|  2011-05-03|
|  4|536365| 84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United King

In [43]:
# This function converts the string cell into a date:
func =  udf(lambda x: datetime.strptime(x, '%Y-%m-%d'), DateType())
func_adms = udf(lambda x: datetime.strptime(x, '%m/%d/%Y %H:%M'), DateType())

df2 = df.withColumn('CRET_DTIME_M2', func(col('CRET_DTIME_M')))
df2 = df2.withColumn('ADMS_DTIME_M2', func_adms(col('ADMS_DTIME_M')))

# This function makes month column by using datetime column
func2 = udf(lambda x: str(x.year) + "-" + "0" + str(x.month) if x.month < 10 else str(x.year) + "-" + str(x.month))

df3 = df2.withColumn('CRET_DTIME_M', func2(col('CRET_DTIME_M2')))
df3 = df3.withColumn('ADMS_DTIME_M', func2(col('ADMS_DTIME_M2')))
df3 = df3.drop(df3.CRET_DTIME_M2)   # 사용한 열 삭제
df3 = df3.drop(df3.ADMS_DTIME_M2)   # 사용한 열 삭제
df3.show()

+---+------+-------+--------------------+--------+------------+---------+----------+--------------+-------------+------------+
|_c0|MBR_ID|MPIC_ID|         Description|Quantity|ADMS_DTIME_M|UnitPrice|CustomerID|       Country|CheckoutPrice|CRET_DTIME_M|
+---+------+-------+--------------------+--------+------------+---------+----------+--------------+-------------+------------+
|  0|536365| 85123A|WHITE HANGING HEA...|       6|     2010-12|     2.55|     17850|United Kingdom|         15.3|     2011-02|
|  1|536365|  71053| WHITE METAL LANTERN|       6|     2010-12|     3.39|     17850|United Kingdom|        20.34|     2010-12|
|  2|536365| 84406B|CREAM CUPID HEART...|       8|     2010-12|     2.75|     17850|United Kingdom|           22|     2011-03|
|  3|536365| 84029G|KNITTED UNION FLA...|       6|     2010-12|     3.39|     17850|United Kingdom|        20.34|     2011-05|
|  4|536365| 84029E|RED WOOLLY HOTTIE...|       6|     2010-12|     3.39|     17850|United Kingdom|        20.3

- MBR_ID: 고객 ID
- MPIC_ID: 인스타 글 ID
- ADMS_DTIME_M: 가입 시기
- CRET_DTIME_M: 업로드 시기

In [44]:
# 원하는 열만 추출
df_new = df3.select(
    col('MBR_ID'),
    col('MPIC_ID'),
    col('ADMS_DTIME_M'),
    col('CRET_DTIME_M')
)
df_new.show()

+------+-------+------------+------------+
|MBR_ID|MPIC_ID|ADMS_DTIME_M|CRET_DTIME_M|
+------+-------+------------+------------+
|536365| 85123A|     2010-12|     2011-02|
|536365|  71053|     2010-12|     2010-12|
|536365| 84406B|     2010-12|     2011-03|
|536365| 84029G|     2010-12|     2011-05|
|536365| 84029E|     2010-12|     2011-01|
|536365|  22752|     2010-12|     2011-11|
|536365|  21730|     2010-12|     2010-11|
|536366|  22633|     2010-12|     2011-10|
|536366|  22632|     2010-12|     2010-12|
|536367|  84879|     2010-12|     2011-09|
|536367|  22745|     2010-12|     2010-11|
|536367|  22748|     2010-12|     2011-05|
|536367|  22749|     2010-12|     2011-03|
|536367|  22310|     2010-12|     2011-08|
|536367|  84969|     2010-12|     2011-11|
|536367|  22623|     2010-12|     2011-04|
|536367|  22622|     2010-12|     2011-11|
|536367|  21754|     2010-12|     2011-06|
|536367|  21755|     2010-12|     2011-09|
|536367|  21777|     2010-12|     2011-05|
+------+---

In [45]:
df_new.groupby('CRET_DTIME_M').count().sort(['CRET_DTIME_M']).show()

+------------+-----+
|CRET_DTIME_M|count|
+------------+-----+
|     2010-11|28546|
|     2010-12|29802|
|     2011-01|29770|
|     2011-02|26842|
|     2011-03|29704|
|     2011-04|28726|
|     2011-05|29730|
|     2011-06|28691|
|     2011-07|29659|
|     2011-08|29921|
|     2011-09|28500|
|     2011-10|29833|
|     2011-11|28953|
|     2011-12|19207|
+------------+-----+



In [46]:
df_new.groupby('CRET_DTIME_M').count().sort(['CRET_DTIME_M']).filter((df_new.CRET_DTIME_M >= '2011-03') & (df_new.CRET_DTIME_M <= '2011-10')).show()

+------------+-----+
|CRET_DTIME_M|count|
+------------+-----+
|     2011-03|29704|
|     2011-04|28726|
|     2011-05|29730|
|     2011-06|28691|
|     2011-07|29659|
|     2011-08|29921|
|     2011-09|28500|
|     2011-10|29833|
+------------+-----+



In [35]:
df_new_cnt = df_new.groupby('CRET_DTIME_M').agg(countDistinct(col('MBR_ID')).alias('인스타 업로드 회원수'))
df_new_cnt = df_new_cnt.sort(df_new_cnt.CRET_DTIME_M.asc())
df_new_cnt = df_new_cnt.filter((df_new_cnt.CRET_DTIME_M >= '2011-03') & (df_new_cnt.CRET_DTIME_M <= '2011-10'))
df_new_cnt.show()

+------------+--------------------+
|CRET_DTIME_M|인스타 업로드 회원수|
+------------+--------------------+
|     2011-03|               11600|
|     2011-04|               11405|
|     2011-05|               11553|
|     2011-06|               11425|
|     2011-07|               11555|
|     2011-08|               11587|
|     2011-09|               11346|
|     2011-10|               11550|
+------------+--------------------+



## Q2. 가입 기준 업로드 회원

### 1. 날짜 조건 적용

In [137]:
# 가입 기준 3~10월
filter_admin = (df_new.ADMS_DTIME_M >= '2011-03') & (df_new.ADMS_DTIME_M <= '2011-10')
df_filter_admin = df_new.filter(filter_admin)
df_filter_admin = df_filter_admin.sort(['ADMS_DTIME_M'])

# 업로드 기준 10월 이내
filter_upload = df_filter_admin.CRET_DTIME_M <= '2011-10'
df_filter_all = df_filter_admin.filter(filter_upload)
df_filter_all.show(10)

+------+-------+------------+------------+
|MBR_ID|MPIC_ID|ADMS_DTIME_M|CRET_DTIME_M|
+------+-------+------------+------------+
|545220|  22191|     2011-03|     2011-08|
|545220|  21485|     2011-03|     2011-07|
|545220|  21700|     2011-03|     2011-09|
|545220|  22556|     2011-03|     2011-05|
|545220| 84029G|     2011-03|     2011-01|
|545220| 17091J|     2011-03|     2010-11|
|545220|  84678|     2011-03|     2011-06|
|545220|  21955|     2011-03|     2011-07|
|545220|  48194|     2011-03|     2011-08|
|545220|  22139|     2011-03|     2011-06|
+------+-------+------------+------------+
only showing top 10 rows



### 2. 월별, ID 별로 업로드를 매월 했는지 check
- 가입일자, ID 기준으로 그룹핑
- 업로드를 매월 한번이라도 했으면 True 

In [125]:
df_upload_bool = df_filter_all.groupby('ADMS_DTIME_M','MBR_ID').agg(
    countDistinct(when(col('CRET_DTIME_M') == '2011-03', True)).alias('cnt_03'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-04', True)).alias('cnt_04'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-05', True)).alias('cnt_05'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-06', True)).alias('cnt_06'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-07', True)).alias('cnt_07'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-08', True)).alias('cnt_08'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-09', True)).alias('cnt_09'),
    countDistinct(when(col('CRET_DTIME_M') == '2011-10', True)).alias('cnt_10'),)
df_upload_bool.show()

+------------+------+------+------+------+------+------+------+------+------+
|ADMS_DTIME_M|MBR_ID|cnt_03|cnt_04|cnt_05|cnt_06|cnt_07|cnt_08|cnt_09|cnt_10|
+------------+------+------+------+------+------+------+------+------+------+
|     2011-03|546677|     1|     0|     0|     0|     0|     0|     0|     0|
|     2011-03|546909|     1|     1|     1|     0|     1|     1|     1|     1|
|     2011-04|549957|     1|     0|     0|     1|     1|     1|     0|     1|
|     2011-05|552969|     1|     1|     0|     0|     0|     1|     1|     1|
|     2011-06|555281|     0|     0|     0|     0|     1|     0|     0|     0|
|     2011-06|556234|     0|     1|     0|     0|     1|     1|     0|     1|
|     2011-07|559509|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-04|551400|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-07|559541|     0|     0|     0|     0|     0|     0|     0|     0|
|     2011-03|545524|     0|     1|     1|     0|     0|     1| 

### 3. 매월별로 모두 구매를 한 고객만 추출

In [133]:
df_upload_all = df_upload_bool.filter(
    (df_upload_bool.cnt_03 == True) &
    (df_upload_bool.cnt_04 == True) &
    (df_upload_bool.cnt_05 == True) &
    (df_upload_bool.cnt_06 == True) &
    (df_upload_bool.cnt_07 == True) &
    (df_upload_bool.cnt_08 == True) &
    (df_upload_bool.cnt_09 == True) &
    (df_upload_bool.cnt_10 == True))
df_upload_all.show()                   

+------------+------+------+------+------+------+------+------+------+------+
|ADMS_DTIME_M|MBR_ID|cnt_03|cnt_04|cnt_05|cnt_06|cnt_07|cnt_08|cnt_09|cnt_10|
+------------+------+------+------+------+------+------+------+------+------+
|     2011-07|559509|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-04|551400|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-03|547102|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-05|551584|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-03|547669|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-04|549129|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-04|548664|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-05|554812|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-10|570388|     1|     1|     1|     1|     1|     1|     1|     1|
|     2011-10|571404|     1|     1|     1|     1|     1|     1| 

In [135]:
# 전체 row 개수
df_upload_all.count()

2371

### 4. 가입 기준으로 그룹핑 후, 가입 기준으로 오름차순 정렬

In [136]:
# 가입 기준으로 그룹핑
df_gb = df_upload_all.groupby('ADMS_DTIME_M').agg(countDistinct(col('MBR_ID')).alias('가입기준 업로드 회원수'))
df_gb = df_gb.sort(['ADMS_DTIME_M'])
df_gb.show()

+------------+----------------------+
|ADMS_DTIME_M|가입기준 업로드 회원수|
+------------+----------------------+
|     2011-03|                   249|
|     2011-04|                   219|
|     2011-05|                   281|
|     2011-06|                   257|
|     2011-07|                   252|
|     2011-08|                   257|
|     2011-09|                   382|
|     2011-10|                   474|
+------------+----------------------+

